In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from itertools import product
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima


In [2]:
data_df = pd.DataFrame({})
cpi = pd.read_csv(r'/Users/dhruvpandit/Documents/GitHub/Projects/Datasets/Inflation/CPHPTT01EZM659N.csv', index_col='DATE').rename(columns={'CPHPTT01EZM659N' : 'CPI'})
oil = pd.read_csv(r'/Users/dhruvpandit/Documents/GitHub/Projects/Datasets/Inflation/MCOILBRENTEU.csv', index_col='DATE').rename(columns={'MCOILBRENTEU' : 'OIL'})
gas = pd.read_csv(r'/Users/dhruvpandit/Documents/GitHub/Projects/Datasets/Inflation/PNGASEUUSDM.csv', index_col='DATE').rename(columns={'PNGASEUUSDM' : 'GAS'})
longterm = pd.read_csv(r'/Users/dhruvpandit/Documents/GitHub/Projects/Datasets/Inflation/IRLTLT01EZM156N.csv', index_col='DATE').rename(columns={'IRLTLT01EZM156N' : 'LongTerm'})
threemonth = pd.read_csv(r'/Users/dhruvpandit/Documents/GitHub/Projects/Datasets/Inflation/IR3TIB01EZM156N.csv', index_col='DATE').rename(columns={'IR3TIB01EZM156N' : 'ThreeMonth'})

In [3]:
cpi = pd.Series(cpi['CPI'])
oil = pd.Series(oil['OIL'])
gas = pd.Series(gas['GAS'])
longterm = pd.Series(longterm['LongTerm'])
threemonth = pd.Series(threemonth['ThreeMonth'])

In [4]:
exog = pd.DataFrame({
    'oil' : oil.values, 
    'gas' : gas.values,
    'lt' : longterm.values, 
    'three' : threemonth.values
}).set_index(pd.Index(oil.index))
exog

,oil,gas,lt,three
DATE,,,,
1997-01-01,23.54,3.000000,6.2300,4.3900
1997-02-01,20.85,2.944444,6.0600,4.4300
1997-03-01,19.13,2.760000,6.3200,4.5000
1997-04-01,17.56,2.730000,6.3900,4.3900
1997-05-01,19.02,2.730000,6.1800,4.3000
...,...,...,...,...
2022-08-01,100.45,69.977239,2.0583,0.3947
2022-09-01,89.76,55.179127,2.8138,1.0109
2022-10-01,93.33,20.806637,3.1850,1.4277


In [5]:
from statsmodels.tsa.stattools import zivot_andrews
zivot_andrews(cpi)


(-2.4456225191318075,
 0.9874673475822978,
 {'1%': -5.27644, '5%': -4.81067, '10%': -4.56618},
 12,
 158)

In [43]:
auto_arima(y=cpi, X = exog, m = 12, seasonal=True)

ARIMA(order=(3, 0, 2), scoring_args={}, seasonal_order=(0, 0, 1, 12),
      suppress_warnings=True)

In [7]:
train_cpi = cpi[:308]
test_cpi = cpi[308:]
train_exog = exog[:308]
test_exog = exog[308:]


In [47]:
model_exog = SARIMAX(
    train_cpi, exog=train_exog, order=(3, 0, 2), seasonal_order=(0, 0, 1, 12)
)
res = model_exog.fit()
start = len(train_cpi)
end = len(train_cpi) + len(test_cpi) -1

# ax = test_cpi.plot(legend=True, figsize=(16,8))
# prediction.plot(legend=True)

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.16849D+00    |proj g|=  3.92830D+01

At iterate    5    f=  7.73680D-01    |proj g|=  1.23142D+00

At iterate   10    f=  1.52113D-01    |proj g|=  1.64784D+00

At iterate   15    f= -1.24797D-02    |proj g|=  2.59632D-01

At iterate   20    f= -2.82391D-02    |proj g|=  5.28073D-02

At iterate   25    f= -3.03434D-02    |proj g|=  7.07212D-02

At iterate   30    f= -3.06200D-02    |proj g|=  1.01669D-01

At iterate   35    f= -3.12398D-02    |proj g|=  6.33454D-02

At iterate   40    f= -3.12668D-02    |proj g|=  7.41877D-03

At iterate   45    f= -3.14183D-02    |proj g|=  2.45848D-01

At iterate   50    f= -3.15309D-02    |proj g|=  2.75903D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [41]:
pred = res.predict(0, end, exog = test_exog).rename('Pred')

In [37]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=cpi.index, y=cpi.values,
                    mode='lines',
                    name='Real CPI'))
fig.add_trace(go.Scatter(x=pred.index, y=pred.values,
                    mode='lines',
                    name='Predicted CPI'))
fig.update_layout(title='Monthly Eurozone CPI From 1997 to 2022 (Blue) with Predicted CPI Using Exogenous Variables',
                   xaxis_title='Month',
                   yaxis_title='CPI Rate')

fig.show()

In [42]:
auto_arima(y=cpi, m = 12, seasonal=True)

ARIMA(order=(2, 0, 1), scoring_args={}, seasonal_order=(0, 0, 1, 12),
      suppress_warnings=True)

In [46]:
model_2 = SARIMAX(
    train_cpi, order=(2, 0, 1), seasonal_order=(0, 0, 1, 12)
)
res2 = model_2.fit()
start2 = len(train_cpi)
end2 = len(train_cpi) + len(test_cpi) -1

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.99963D+02    |proj g|=  5.10996D+03

At iterate    5    f=  2.81796D+00    |proj g|=  2.18910D+00

At iterate   10    f=  4.85500D-01    |proj g|=  4.33496D-01

At iterate   15    f=  1.39576D-01    |proj g|=  6.13563D-02

At iterate   20    f=  1.20899D-01    |proj g|=  1.39277D-01

At iterate   25    f=  1.08351D-01    |proj g|=  1.98651D-02

At iterate   30    f=  1.07693D-01    |proj g|=  2.20521D-03

At iterate   35    f=  1.07664D-01    |proj g|=  2.49395D-04

At iterate   40    f=  1.07664D-01    |proj g|=  4.90122D-04

At iterate   45    f=  1.07659D-01    |proj g|=  8.34906D-03

At iterate   50    f=  1.07542D-01    |proj g|=  3.59865D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [48]:
pred2 = res2.predict(0, end, exog = test_exog).rename('Pred')
forecast = res2.get_forecast(steps = (12))

In [52]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cpi.index, y=cpi.values,
                    mode='lines',
                    name='Real CPI'))
fig.add_trace(go.Scatter(x=pred2.index, y=pred2.values,
                    mode='lines',
                    name='Predicted CPI'))
fig.add_trace(go.Scatter(x=forecast.predicted_mean.index, y=forecast.predicted_mean.values,
                    mode='lines',
                    name='Forecasted CPI'))
fig.update_layout(title='Monthly Eurozone CPI From 1997 to 2022 with Predicted CPI (Red) and Forecasted CPI (Green)',
                   xaxis_title='Month',
                   yaxis_title='CPI Rate')

fig.show()

In [53]:
from statsmodels.tools.eval_measures import rmse

rmse1 = rmse(pred.values, cpi.values)
rmse2 = rmse(pred2.values, cpi.values)
print(rmse1)
print(rmse2)


0.33454871429313093
0.34532783977115183
